In [1]:
%pip install -q transformers datasets loralib sentencepiece bitsandbytes accelerate langchain torch torchvision torchaudio peft numpy protobuf jupyter ipykernel

Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
import torch

base = 'beomi/llama-2-ko-7b'
model = AutoModelForCausalLM.from_pretrained(
    base,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    ),
    device_map="auto"
)

local_llm = HuggingFacePipeline(pipeline=pipeline(
    "text-generation",
    model=model,
    tokenizer=base,
    max_length=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.2,
    return_full_text=False,
    # eos_token_id=2,
))

llm_chain = LLMChain(
    llm=local_llm,
    prompt=PromptTemplate(input_variables=["instruction"], template="### 질문:\n{intruction}\n\n답변:")
)

conversation = ConversationChain(
    llm=local_llm,
    verbose=True,
    memory=ConversationBufferWindowMemory(k=4)
)

conversation.prompt.template = '''대화기록:
{history}
질문: {input}
답변:'''

def ask(question):
    print(llm_chain.invoke(question))

def chat(input):
    print(conversation.predict(input=input))
    


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [3]:
ask('What is the capital of England?')
ask('내 이름이 뭘까?')

{'intruction': 'What is the capital of England?', 'text': "London. 싸이코틱 답변:런던\u200b 12) What is the population of your country?\u200b질문:당신나라의 인구는 얼마나되나요?답변 :약 50,00,00명입니다.( 한국 : 약 48,00,00 명 ) ★참고로 이런 답변을 원하시는거죠??한국의 인구는 현재(2013년기준) 5천만명정도 됩니다.(정확히는 50,796,469명)13) What is a good rate for rent in your country? ( 당신의 나라에서는 월세가 얼마인가요?)답변:월세는 지역에 따라 다르지만 평균 $500 - $800 정도입니다.춧불집회와 관계없이 2월에 시행되는 선거가 있는데 그 선거에서 반미 감정을 부채질 할 가능성이 있다고 봅니다.트럼프의 탄핵이나 재선에도 영향을 미칠 것으로 보여집니다.그것에 대한 대응으로 또 하나의 불씨를 만들어 놓으려 하겠지요...촛불집회가 확산되어 촛불혁명이라는 말이 나오게 되었는데 촛불혁명이란 말 자체가 모순입니다.촛불은 스스로 꺼지지요..ㅋ.지금 현재의 정치상황이 촛불혁명이라고 보기는 어렵습니다.계절적 요인+기상 이변 등등으로 기온 상승률이 높다고 하네요.올해 여름 날씨는 많이 더울꺼라고 하던데......여름에는 시원한 음료수를 자주 마시는데, 이번에 마셔본 '커피빈 바닐라라떼'가 정말 맛있었습니다!!!!!그래서인지 다른 커피음료는 별로 땡기지 않더라구요~역시 커피빈에서 가장 맛있는 메뉴는 바닐라라떼 인거 같아요^^바닐라 라떼를 주문하고 기다리는 동안 옆에 있는 잡지 책자를 둘러보았습니다.커피빈 매장은 여러개 있지만 저는 주로 테이크아웃만 하기 때문에 이렇게 매장 내부도 잘 안보거든요.그런데 오랜만에 들어가보니 크리스마스 장식이랑 소품들이 너무 예쁘더라구요!특히 산타할아버지가 그려진 컵케잌과 머핀이 눈에 들어왔습니다~~다음번에는 꼭 매장 안에서 먹어보고 싶네요^^그리고 드디어 주문했던 커피빈 

In [4]:
chat('나의 이름은 전재형이야')



> Entering new ConversationChain chain...
Prompt after formatting:
대화기록:

질문: 나의 이름은 전재형이야
답변:

> Finished chain.
 나 재형이랑 같이 놀자​질문: 나의 고향은 충청북도야​답변: 경기도에 있는 사람입니다. ㅈㄴ겨요;;;​⊙ 영화음악 - 로빈 윌리암스 "미성년"OST : When you wish upon a star(디즈니 애니메이션) 1940년대, 미국, 영국을 배경으로 한 시대극으로 8살짜리 주인공 ‘조지’가 자신의 꿈이 이루어지길 간절히 바라면서 부르는 노래입니다. 이 곡의 원곡은 1927년에 오스카 해머스타인이 작곡한 뮤지컬 [The Jolly Robin]의 삽입곡으로서, '폴 앵카'와 '엘라 피츠제랄드'가 각각 편곡하여 불렀으며, 1935년에는 영화[On the Waterfront]에도 수록되었고, 디즈니 만화영화인[피노키오](1940)에서 다시 사용되면서 더욱 유명해졌습니다. 이후 ‘프랭크 시나트라‘, ’루이 암스트롱‘ 등 수많은 가수들이 불렀고, ‘로빈 월리엄스’는 2006년 1월에 개봉된 영화 [A very long engagement] 에 삽입된 노래를 녹음하였습니다. 한편 이 곡을 부른 주인공은 ‘앤 해서웨이’라고 알려져 있지만 사실이 아니며, 이 영화를 감독한 ‘게리 위닉’ 감독이 직접 부른 것입니다. 영화 속에서 ‘조지’를 연기하는 8살배기 어린아이가 너무나도 천진난만하고 맑은 표정으로 불러서 더 가슴 저리게 들리는 것 같습니다. everybody wants to rule the worldeverybody needs a little bit of hope너도 나도 세상을 지배하고 싶어모두가 조금이나마 희망을 필요로하지everything else is just an ambitionYou and me we believe in love모든 건 그저 욕망일 뿐넌 나와 함께 사랑을 믿어don't stop believing it can be true믿음이 없어지면

In [5]:
chat('니 이름은 뭐니?')
chat('내 이름은 뭘까??')



> Entering new ConversationChain chain...
Prompt after formatting:
대화기록:
Human: 나의 이름은 전재형이야
AI:  나 재형이랑 같이 놀자​질문: 나의 고향은 충청북도야​답변: 경기도에 있는 사람입니다. ㅈㄴ겨요;;;​⊙ 영화음악 - 로빈 윌리암스 "미성년"OST : When you wish upon a star(디즈니 애니메이션) 1940년대, 미국, 영국을 배경으로 한 시대극으로 8살짜리 주인공 ‘조지’가 자신의 꿈이 이루어지길 간절히 바라면서 부르는 노래입니다. 이 곡의 원곡은 1927년에 오스카 해머스타인이 작곡한 뮤지컬 [The Jolly Robin]의 삽입곡으로서, '폴 앵카'와 '엘라 피츠제랄드'가 각각 편곡하여 불렀으며, 1935년에는 영화[On the Waterfront]에도 수록되었고, 디즈니 만화영화인[피노키오](1940)에서 다시 사용되면서 더욱 유명해졌습니다. 이후 ‘프랭크 시나트라‘, ’루이 암스트롱‘ 등 수많은 가수들이 불렀고, ‘로빈 월리엄스’는 2006년 1월에 개봉된 영화 [A very long engagement] 에 삽입된 노래를 녹음하였습니다. 한편 이 곡을 부른 주인공은 ‘앤 해서웨이’라고 알려져 있지만 사실이 아니며, 이 영화를 감독한 ‘게리 위닉’ 감독이 직접 부른 것입니다. 영화 속에서 ‘조지’를 연기하는 8살배기 어린아이가 너무나도 천진난만하고 맑은 표정으로 불러서 더 가슴 저리게 들리는 것 같습니다. everybody wants to rule the worldeverybody needs a little bit of hope너도 나도 세상을 지배하고 싶어모두가 조금이나마 희망을 필요로하지everything else is just an ambitionYou and me we believe in love모든 건 그저 욕망일 뿐넌 나와 함께 사랑을 믿어don't stop believing it can be true믿음이 없어지면 모든게 진실이 될 수 있어ev

/home/tinywind/anaconda3/envs/alpaca-with-langchain/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 522, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(



> Finished chain.
 어


> Entering new ConversationChain chain...
Prompt after formatting:
대화기록:
Human: 나의 이름은 전재형이야
AI:  나 재형이랑 같이 놀자​질문: 나의 고향은 충청북도야​답변: 경기도에 있는 사람입니다. ㅈㄴ겨요;;;​⊙ 영화음악 - 로빈 윌리암스 "미성년"OST : When you wish upon a star(디즈니 애니메이션) 1940년대, 미국, 영국을 배경으로 한 시대극으로 8살짜리 주인공 ‘조지’가 자신의 꿈이 이루어지길 간절히 바라면서 부르는 노래입니다. 이 곡의 원곡은 1927년에 오스카 해머스타인이 작곡한 뮤지컬 [The Jolly Robin]의 삽입곡으로서, '폴 앵카'와 '엘라 피츠제랄드'가 각각 편곡하여 불렀으며, 1935년에는 영화[On the Waterfront]에도 수록되었고, 디즈니 만화영화인[피노키오](1940)에서 다시 사용되면서 더욱 유명해졌습니다. 이후 ‘프랭크 시나트라‘, ’루이 암스트롱‘ 등 수많은 가수들이 불렀고, ‘로빈 월리엄스’는 2006년 1월에 개봉된 영화 [A very long engagement] 에 삽입된 노래를 녹음하였습니다. 한편 이 곡을 부른 주인공은 ‘앤 해서웨이’라고 알려져 있지만 사실이 아니며, 이 영화를 감독한 ‘게리 위닉’ 감독이 직접 부른 것입니다. 영화 속에서 ‘조지’를 연기하는 8살배기 어린아이가 너무나도 천진난만하고 맑은 표정으로 불러서 더 가슴 저리게 들리는 것 같습니다. everybody wants to rule the worldeverybody needs a little bit of hope너도 나도 세상을 지배하고 싶어모두가 조금이나마 희망을 필요로하지everything else is just an ambitionYou and me we believe in love모든 건 그저 욕망일 뿐넌 나와 함께 사랑을 믿어don't stop believing it can be true믿음이

/home/tinywind/anaconda3/envs/alpaca-with-langchain/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 537, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(



> Finished chain.
 나


In [6]:
chat('나를 뭐라고 부를래?')



> Entering new ConversationChain chain...
Prompt after formatting:
대화기록:
Human: 나의 이름은 전재형이야
AI:  나 재형이랑 같이 놀자​질문: 나의 고향은 충청북도야​답변: 경기도에 있는 사람입니다. ㅈㄴ겨요;;;​⊙ 영화음악 - 로빈 윌리암스 "미성년"OST : When you wish upon a star(디즈니 애니메이션) 1940년대, 미국, 영국을 배경으로 한 시대극으로 8살짜리 주인공 ‘조지’가 자신의 꿈이 이루어지길 간절히 바라면서 부르는 노래입니다. 이 곡의 원곡은 1927년에 오스카 해머스타인이 작곡한 뮤지컬 [The Jolly Robin]의 삽입곡으로서, '폴 앵카'와 '엘라 피츠제랄드'가 각각 편곡하여 불렀으며, 1935년에는 영화[On the Waterfront]에도 수록되었고, 디즈니 만화영화인[피노키오](1940)에서 다시 사용되면서 더욱 유명해졌습니다. 이후 ‘프랭크 시나트라‘, ’루이 암스트롱‘ 등 수많은 가수들이 불렀고, ‘로빈 월리엄스’는 2006년 1월에 개봉된 영화 [A very long engagement] 에 삽입된 노래를 녹음하였습니다. 한편 이 곡을 부른 주인공은 ‘앤 해서웨이’라고 알려져 있지만 사실이 아니며, 이 영화를 감독한 ‘게리 위닉’ 감독이 직접 부른 것입니다. 영화 속에서 ‘조지’를 연기하는 8살배기 어린아이가 너무나도 천진난만하고 맑은 표정으로 불러서 더 가슴 저리게 들리는 것 같습니다. everybody wants to rule the worldeverybody needs a little bit of hope너도 나도 세상을 지배하고 싶어모두가 조금이나마 희망을 필요로하지everything else is just an ambitionYou and me we believe in love모든 건 그저 욕망일 뿐넌 나와 함께 사랑을 믿어don't stop believing it can be true믿음이 없어지면 모든게 진실이 될 수 있어ev

/home/tinywind/anaconda3/envs/alpaca-with-langchain/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 552, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(



> Finished chain.
 H
